In [1]:
import pandas as pd
from datetime import date, timedelta, datetime
import time
import os 

def fama_french_files_create_date(inp_df): #ASSUMES MONTH YEAR DATE COLUMN IS DM
    lastrow =  inp_df.loc[inp_df["DM"] == " Annual Factors: January-December"].index[0]
    df = inp_df.iloc[:lastrow,:]
    df["Date"] = df["DM"].str.strip() + "01"
    df["Date"] = [datetime.strptime(dt, "%Y%m%d") for dt in df["Date"]]
    df["month_year"] = df["Date"].dt.to_period("M")
    df.drop(columns = ["DM","Date"], inplace = True)
    for col in df.columns:
        if col != "month_year": 
            df[col] = pd.to_numeric(df[col])
    return df

cwd = os.getcwd()

ff_3f_filename = os.path.join(cwd,"Japan_5_Factors.csv")
ff_3f_raw = pd.read_csv(ff_3f_filename, skiprows = 6)
ff_3f_cols = ["DM", "Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]
ff_3f_raw.columns = ff_3f_cols
ff_3f_sta = fama_french_files_create_date(inp_df = ff_3f_raw)

ff_mom_filename = os.path.join(cwd,"Japan_MOM_Factor.csv")
ff_mom_raw = pd.read_csv(ff_mom_filename, skiprows = 6)
ff_mom_cols = ["DM", "WML"]
ff_mom_raw.columns = ff_mom_cols
ff_mom_sta = fama_french_files_create_date(inp_df = ff_mom_raw)

ff_fin = pd.merge(ff_3f_sta, ff_mom_sta, how = "left", left_on = "month_year", right_on = "month_year")


C:\Users\neera\AppData\Local\Temp\ipykernel_18612\3757166401.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = df["DM"].str.strip() + "01"
C:\Users\neera\AppData\Local\Temp\ipykernel_18612\3757166401.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = [datetime.strptime(dt, "%Y%m%d") for dt in df["Date"]]
C:\Users\neera\AppData\Local\Temp\ipykernel_18612\3757166401.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

get fama french returns

In [14]:
pel_filename = "pelargos_returns.csv"
pel_raw = pd.read_csv(pel_filename)
pel_raw["month_year"] = [datetime.strptime(dt, "%d/%m/%Y") for dt in pel_raw["date"]]
pel_raw["month_year"] = pel_raw["month_year"].dt.to_period("M")
pel_raw.drop(columns = ["date"], inplace = True)

get pelargos returns

In [15]:
reg_df = pel_raw.merge(ff_fin,how="left",left_on="month_year",right_on="month_year")
ff_cols = ["Mkt-RF", "SMB", "HML"]
ch_cols = ["Mkt-RF", "SMB", "HML", "WML"]
pel_col = "pelargos"

regr_ff = nwest(inp_Y = reg_df[pel_col], inp_X = reg_df[ff_cols], inp_modelname = pel_col + " FamaFrench 3 Factor model")
regr_ch = nwest(inp_Y = reg_df[pel_col], inp_X = reg_df[ch_cols], inp_modelname = pel_col + " Carhart 4 Factor model")